In [1]:
import numpy as np                                                                                                                                             
import pandas as pd                                                                                                                                            
import tensorflow as tf                                                                                                                                        
from sklearn.preprocessing import StandardScaler                                                                                                               
from tqdm import tqdm
import matplotlib.pyplot as plt
                                                                                                                                                               
import os                                                                                                                                                      
os.environ['CUDA_VISIBLE_DEVICES']="3"
physical_devices = tf.config.list_physical_devices('GPU')                                                                                                      
tf.config.experimental.set_memory_growth(physical_devices[0], True)                                                                                            
                                                                                                                                                               
mc = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Rapgap_nominal.pkl")                                                                           
data = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Data_nominal.pkl") 

In [2]:
#Apply Cuts                                                                                                                                                    
cut_subleading_jet = True                                                                                                                                      
if cut_subleading_jet:                                                                                                                                         
    mc = mc.loc[(slice(None),0), :]                                                                                                                            
    data = data.loc[(slice(None),0), :]                                                                                                                        
                                                                                                                                                               
pass_fiducial = np.array(mc['pass_fiducial'])                                                                                                                  
pass_truth = np.array(mc['pass_truth'])                                                                                                                        
                                                                                                                                                               
theta0_G = mc[['gene_px','gene_py','gene_pz','genjet_pt','genjet_eta','genjet_phi','genjet_dphi','genjet_qtnorm']].to_numpy()
theta_unknown_S = data[['e_px','e_py','e_pz','jet_pt','jet_eta','jet_phi','jet_dphi','jet_qtnorm']].to_numpy()

#Get StandardScalar from raw data, apply to Rapgap in each iteration later                                                                                     
scaler_data = StandardScaler()                                                                                                                                 
scaler_data.fit(theta_unknown_S)                                                                                                                               
print("\n Length of theta0G =",len(theta0_G),"\n")                                                                                                        
                                                                                                                                                                                                                      


 Length of theta0G = 54120771 



In [3]:
N_Iter = 18 #technically 0-19, but boot 4 failed the last one...
N_Boots = 5 #1-4. Fencepost!
path = "/clusterfs/ml4hep_nvme2/ftoralesacosta/h1_check/h1_lepton_jet_asymmetry/models/npy_20_iterations_5boots"


In [4]:
#Check the weights work as expected for one bootstrap
for i in [1,2,3,4,5,10,15,19]:
    weights = np.load(f"{path}/Rapgap_bootstrap_{1}_NNweights_step2_iteration_{i}.npy")
    plt.hist(weights,label=f"Itertation {i}",bins=np.linspace(0,2,100))
    plt.legend()

FileNotFoundError: [Errno 2] No such file or directory: '/clusterfs/ml4hep_nvme2/ftoralesacosta/h1_check/h1_lepton_jet_asymmetry/models/npy_20_iterations_5boots/Rapgap_bootstrap_1_NNweights_step2_iteration_1.npy'

In [ ]:
#Load the first ensemble of bootstrap weights
boot_ensemble = []
for boot in range(1,N_Boots):
    print(boot)
    nn_weights_step2 = np.load(f"{path}/Rapgap_bootstrap_{boot}_NNweights_step2_iteration_{N_Iter}.npy")
    boot_ensemble.append(nn_weights_step2)
    plt.hist(nn_weights_step2,label=f"Bootstrap {boot}",bins=np.linspace(0,2,100),alpha=0.5)
np.save("Five_Bootstraps_20Iterations.npy",boot_ensemble)

In [ ]:
boot_ensemble = np.asarray(boot_ensemble)

## Now Let's try extracting weights from models at the end of training, see if they'er usable...

In [ ]:
# run_iter = 4            
path = "/clusterfs/ml4hep_nvme2/ftoralesacosta/h1_check/h1_lepton_jet_asymmetry/models/last_iteration_only"
bootstrap_model_weights = []                                                                                                 
for boot in tqdm(range(1,5)):

    #Make sure to reset weights                                                                                        
    NNweights_step2 = np.ones(len(theta0_G))                                                                           
    NNweights_step2_hold = np.ones(len(theta0_G))                                                                      
                                                                                                                       
    for run_iter in range(1):                                                                                          
        print(f"{path}/Rapgap_bootstrap_{boot}" )                                                               
                                                                                                                       
        mymodel = tf.keras.models.load_model(                                                                          
             f"{path}/Rapgap_bootstrap_{boot}",compile=False)                                                                                          
                                                                                                                       
        NNweights_step2_hold = mymodel.predict(scaler_data.transform(theta0_G),batch_size=10000)                       
        NNweights_step2_hold = NNweights_step2_hold/(1.-NNweights_step2_hold)                                          
        NNweights_step2_hold = NNweights_step2_hold[:,0]                                                               
        NNweights_step2_hold = np.squeeze(np.nan_to_num(NNweights_step2_hold,posinf=1))                                
        NNweights_step2_hold[pass_truth==0] = 1.                                                                       
        NNweights_step2 = NNweights_step2_hold*NNweights_step2                                                         
                                                                                                                       
        tf.keras.backend.clear_session()                                                                               
                                                                                                                       
    bootstrap_model_weights.append(NNweights_step2)                                                                          
                                                                                                                       
bootstrap_model_weights = np.asarray(bootstrap_model_weights)                                                                      
np.save("bootsrap_lastIter_weights.npy",bootstrap_model_weights) 

In [ ]:
_ = plt.hist(bootstrap_model_weights[0],bins=np.linspace(0,2,100))

## Those models should have been saved along with the previous iterations. These are unusable by themselves!

## Switching to loading ~5 other models, this time with iterations

In [ ]:
N_Iter = 20 #0-19
seeds = [8762,5986,2919,2134,2838]
path = "/clusterfs/ml4hep_nvme2/ftoralesacosta/h1_check/h1_lepton_jet_asymmetry/models"

bootstrap_model_weights = []
for boot,seed in enumerate(seeds):
    #Make sure to reset weights                                                                                        
    NNweights_step2 = np.ones(len(theta0_G))                                                                           
    NNweights_step2_hold = np.ones(len(theta0_G))
    
    for it in range(N_Iter):
        model_string = f"{path}/Rapgap_bootstrap_{boot+1}_{seed}Iteration_{it}model"
                
        print("Loading "+model_string)                                                               
                                                                                                                       
        mymodel = tf.keras.models.load_model(                                                                          
             f"{model_string}",compile=False)                                                                                          
                                                                                                                       
        NNweights_step2_hold = mymodel.predict(scaler_data.transform(theta0_G),batch_size=10000)                       
        NNweights_step2_hold = NNweights_step2_hold/(1.-NNweights_step2_hold)                                          
        NNweights_step2_hold = NNweights_step2_hold[:,0]                                                               
        NNweights_step2_hold = np.squeeze(np.nan_to_num(NNweights_step2_hold,posinf=1))                                
        NNweights_step2_hold[pass_truth==0] = 1.                                                                       
        NNweights_step2 = NNweights_step2_hold*NNweights_step2
        
        tf.keras.backend.clear_session()

    bootstrap_model_weights.append(NNweights_step2)                                                                          
                                                                                                                       
bootstrap_model_weights = np.asarray(bootstrap_model_weights)                                                                      
np.save("five_bootsrap_20Iter_weights.npy",bootstrap_model_weights) 

In [ ]:
np.shape(bootstrap_model_weights)

In [ ]:
N_Iter = 20 #0-19
seeds = [7642]
path = "/clusterfs/ml4hep_nvme2/ftoralesacosta/h1_check/h1_lepton_jet_asymmetry/models"

no6_only = []
for boot,seed in enumerate(seeds):
    #Make sure to reset weights                                                                                        
    NNweights_step2 = np.ones(len(theta0_G))                                                                           
    NNweights_step2_hold = np.ones(len(theta0_G))
    
    for it in range(N_Iter):
        model_string = f"{path}/Rapgap_bootstrap_{6}_{seed}Iteration_{it}model"
                
        print("Loading "+model_string)                                                               
                                                                                                                       
        mymodel = tf.keras.models.load_model(                                                                          
             f"{model_string}",compile=False)                                                                                          
                                                                                                                       
        NNweights_step2_hold = mymodel.predict(scaler_data.transform(theta0_G),batch_size=10000)                       
        NNweights_step2_hold = NNweights_step2_hold/(1.-NNweights_step2_hold)                                          
        NNweights_step2_hold = NNweights_step2_hold[:,0]                                                               
        NNweights_step2_hold = np.squeeze(np.nan_to_num(NNweights_step2_hold,posinf=1))                                
        NNweights_step2_hold[pass_truth==0] = 1.                                                                       
        NNweights_step2 = NNweights_step2_hold*NNweights_step2
        
        tf.keras.backend.clear_session()

    no6_only.append(NNweights_step2)                                                                          
                                                                                                                       
no6_only = np.asarray(no6_only)                                                                      


In [ ]:
print(np.shape(bootstrap_model_weights))

In [ ]:
#bootstrap_model_weights = np.concatenate((bootstrap_model_weights,no6_only),axis=0)
np.save("6_bootsrap_20Iter_weights.npy",bootstrap_model_weights) 

In [ ]:
boot_ensemble = np.concatenate((boot_ensemble,bootstrap_model_weights),axis=0)

In [ ]:
print(np.shape(boot_ensemble))

In [ ]:
np.save("10_bootstrap_20Iter_weights.npy",boot_ensemble)